In [4]:
pip install beautifulsoup4


In [30]:
import requests
import json
import csv
import re
from bs4 import BeautifulSoup
phone_page_url = "https://tiki.vn/api/v2/products?limit=48&include=advertisement&aggregations=1&category=1795&page={}&urlKey=dien-thoai"
product_url = "https://tiki.vn/api/v2/products/{}"

product_id_file = "./product-id.txt"
product_data_file = "./product.txt"
product_file = "./product.csv"
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36"}


In [31]:
def crawl_product_id():
    product_list = []
    i = 1
    while (True):
        print("Crawl page: ", i)
        print(phone_page_url.format(i))
        response = requests.get(phone_page_url.format(i), headers=headers)
        
        if (response.status_code != 200):
            break

        products = json.loads(response.text)["data"]

        if (len(products) == 0):
            break

        for product in products:
            product_id = str(product["id"])
            print("Product ID: ", product_id)
            product_list.append(product_id)

        i += 1

    return product_list, i

def save_product_id(product_list=[]):
    file = open(product_id_file, "w+")
    str = "\n".join(product_list)
    file.write(str)
    file.close()
    print("Save file: ", product_id_file)

# crawl product id
product_list, page = crawl_product_id()

print("No. Page: ", page)
print("No. Product ID: ", len(product_list))

# save product id for backup
save_product_id(product_list)

Crawl page:  1
https://tiki.vn/api/v2/products?limit=48&include=advertisement&aggregations=1&category=1795&page=1&urlKey=phone
Product ID:  122074543
Product ID:  123547395
Product ID:  123554528
Product ID:  142667084
Product ID:  172998120
Product ID:  123554518
Product ID:  142545270
Product ID:  123554545
Product ID:  121790468
Product ID:  120450353
Product ID:  123835560
Product ID:  123565891
Product ID:  170580388
Product ID:  112329147
Product ID:  123547396
Product ID:  170497385
Product ID:  57809866
Product ID:  59519631
Product ID:  88255880
Product ID:  170581659
Product ID:  175967811
Product ID:  123345348
Product ID:  128865871
Product ID:  133578350
Product ID:  74489817
Product ID:  103046198
Product ID:  123348908
Product ID:  164937168
Product ID:  123554520
Product ID:  89888994
Product ID:  141122642
Product ID:  127076431
Product ID:  173383634
Product ID:  125590981
Product ID:  168365980
Product ID:  155339564
Product ID:  120295859
Product ID:  113569117
Prod

In [33]:
def crawl_product(product_list=[]):
    product_detail_list = []
    for product_id in product_list:
        response = requests.get(product_url.format(product_id), headers=headers)
        if (response.status_code == 200):
            product_detail_list.append(response.text)
            print("Crawl product: ", product_id, ": ", response.status_code)
    return product_detail_list

flatten_field = [ "badges", "inventory", "categories", "rating_summary", 
                      "brand", "seller_specifications", "current_seller", "other_sellers", 
                      "configurable_options",  "configurable_products", "specifications", "product_links",
                      "services_and_promotions", "promotions", "stock_item", "installment_info" ]

def adjust_product(product):
    e = json.loads(product)
    if not e.get("id", False):
        return None

    for field in flatten_field:
        if field in e:
            e[field] = json.dumps(e[field], ensure_ascii=False).replace('\n','')

    return e

def save_raw_product(product_detail_list=[]):
    file = open(product_data_file, "w+")
    str = "\n".join(product_detail_list)
    file.write(str)
    file.close()
    print("Save file: ", product_data_file)

def load_raw_product():
    file = open(product_data_file, "r")
    return file.readlines()

def save_product_list(product_json_list):
    file = open(product_file, "w")
    csv_writer = csv.writer(file)

    count = 0
    for p in product_json_list:
        if p is not None:
            if count == 0:
                header = p.keys() 
                csv_writer.writerow(header) 
                count += 1
            csv_writer.writerow(p.values())
    file.close()
    print("Save file: ", product_file)



In [34]:
product_list = crawl_product(product_list)

# save product detail for backup
save_raw_product(product_list)

# product_list = load_raw_product()
# flatten detail before converting to csv
product_json_list = [adjust_product(p) for p in product_list]
# save product to csv
save_product_list(product_json_list)

Crawl product:  122074543 :  200
Crawl product:  123547395 :  200
Crawl product:  123554528 :  200
Crawl product:  142667084 :  200
Crawl product:  172998120 :  200
Crawl product:  123554518 :  200
Crawl product:  142545270 :  200
Crawl product:  123554545 :  200
Crawl product:  121790468 :  200
Crawl product:  120450353 :  200
Crawl product:  123835560 :  200
Crawl product:  123565891 :  200
Crawl product:  170580388 :  200
Crawl product:  112329147 :  200
Crawl product:  123547396 :  200
Crawl product:  170497385 :  200
Crawl product:  57809866 :  200
Crawl product:  59519631 :  200
Crawl product:  88255880 :  200
Crawl product:  170581659 :  200
Crawl product:  175967811 :  200
Crawl product:  123345348 :  200
Crawl product:  128865871 :  200
Crawl product:  133578350 :  200
Crawl product:  74489817 :  200
Crawl product:  103046198 :  200
Crawl product:  123348908 :  200
Crawl product:  164937168 :  200
Crawl product:  123554520 :  200
Crawl product:  89888994 :  200
Crawl product: 